In [7]:
using Revise, NeuralNetworkAnalysis, NeuralVerification
const NV = NeuralVerification
const RA = ReachabilityAnalysis
const IA = IntervalArithmetic

IntervalArithmetic

In [6]:
?NV.Layer

```
Layer{F, N}
```

Consists of `weights` and `bias` for linear mapping, and `activation` for nonlinear mapping.

### Fields

  * `weights::Matrix{N}`
  * `bias::Vector{N}`
  * `activation::F`

See also: [`Network`](@ref)


In [95]:
NN = Network([NV.Layer([0.3 0.2; 0.1 0.5], [0.1, 0.2], NV.Sigmoid()), NV.Layer(hcat(3, 5.), [0.], NV.Id())]);

In [106]:
@taylorize function scalar_quadratic!(du, u, p, t)
    xᴶ, xᴾ = u
    du[1] = zero(xᴶ)
    du[2] = xᴶ *(xᴾ - xᴾ^2)
end

function dnn2hybrid(nn::Network, xᴾ₁₀ = IA.Interval(2., 3.),
                                 xᴾ₂₀ = IA.Interval(1., 2.),
                                 u₀ = IA.Interval(0., 0.))
    I1 = IA.Interval(1., 1.)
    xᴾ = [I1, I1] * 0.5
    xᴶ = [xᴾ₁₀, xᴾ₂₀]
    h = []
    for layer in nn.layers
        n = length(layer.bias)
        xᴶm = layer.weights * xᴶ + layer.bias
        if layer.activation == NV.Sigmoid()
            for i=1:n
                X0 = xᴾ[i] × xᴶm[i]
                ivp = @ivp(x' = scalar_quadratic!(x), dim=2, x(0) ∈ X0)
                sol = RA.solve(ivp, tspan=(0., 1.),
                             alg=TMJets(abs_tol=1e-14, orderQ=2, orderT=6));
                push!(h, sol.F.ext[:xv][end][2])
            end
        else
            push!(h, xᴶm)
        end
        println(h)
        xᴾ = h
        h = []
    end
    return xᴾ
end

dnn2hybrid (generic function with 4 methods)

In [107]:
dnn2hybrid(NN)

Any[[0.938147, 1.22112], [0.938632, 1.26939]]
Any[IntervalArithmetic.Interval{Float64}[[11, 19]]]


1-element Array{Any,1}:
 IntervalArithmetic.Interval{Float64}[[11, 19]]

In [74]:
using BenchmarkTools

In [75]:
@btime dnn2hybrid(NN)

  528.968 ms (6632207 allocations: 548.38 MiB)


1-element Array{Any,1}:
 [1.58778, 1.5886]